In [323]:
from faker import Faker
import pandas as pd
from random import randint, choice, uniform
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Generate a list of trader IDs and companies
trader_ids = [fake.random_number(digits=5) for _ in range(100)]  # 100 unique trader IDs
companies = [fake.company() for _ in range(10)]  # 100 unique companies

# Define a list of currency pairs and corresponding spot rate range
currency_data = {
    'EUR/USD': {'spot_rate_range': (1.1, 1.2)},
    'USD/JPY': {'spot_rate_range': (105, 115)},
    'GBP/USD': {'spot_rate_range': (1.3, 1.4)},
    'USD/CHF': {'spot_rate_range': (0.9, 1.0)},
    'USD/CAD': {'spot_rate_range': (1.2, 1.3)},
    'AUD/USD': {'spot_rate_range': (0.7, 0.8)},
    'NZD/USD': {'spot_rate_range': (0.6, 0.7)},
}

# Define the structure of your transaction data
def generate_transaction():
    currency_pair = choice(list(currency_data.keys()))
    buy_currency, sell_currency = currency_pair.split('/')
    rate_data = currency_data[currency_pair]
    buy_sell_indicator = choice(['Buy', 'Sell'])
    quantity = randint(1, 10000) * 10000
    spot_rate = round(fake.pyfloat(right_digits=4, positive=True, min_value=rate_data['spot_rate_range'][0], max_value=rate_data['spot_rate_range'][1]), 4)  # generating a spot rate within the range
    # calculate a forward rate within 1% of the spot rate
    forward_rate = round(spot_rate * uniform(0.99, 1.01), 4)  
    return {
        'TransactionID': fake.unique.random_number(digits=8),
        'TradingDate': fake.date_between(start_date='-1y', end_date='today'),
        'MaturityDate': fake.date_between(start_date='today', end_date='+1y'),
        'ExecutionTime': (datetime.now()-timedelta(seconds=randint(1,86400))).strftime('%H:%M:%S'),
        'InstrumentID': fake.random_number(digits=12),
        'TraderID': choice(trader_ids),  # select a trader ID from the list
        'BuyCurrency': buy_currency,
        'SellCurrency': sell_currency,
        'SpotRate': spot_rate,
        'ForwardRate': forward_rate,
        'BuySellIndicator': buy_sell_indicator,
        'BuyNotional': round(quantity, 2),
        'SellNotional': round(quantity * spot_rate, 2),
        'Counterparty': choice(companies)  # select a company from the list
    }

# Generate n transactions
def generate_transactions(n=100):
    return [generate_transaction() for _ in range(n)]

# Usage
data = generate_transactions(1000)  # generate 10,000 transactions
transactions_df = pd.DataFrame(data)


In [350]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from scipy.stats import norm

# Assume transaction_df is already provided
# Define a time grid
time_grid = list(range(1, 15)) + list(range(14, 91, 7)) + list(range(90, 366, 30))

# Mock functions to fetch volatility and correlation matrix
def fetch_vol_and_correlation(alternative=False):
    if alternative == False:
        volatilities_usd = {
            'EUR': 0.1,
            'JPY': 0.15,
            'GBP': 0.12,
            'CHF': 0.1,
            'CAD': 0.13,
            'AUD': 0.14,
            'NZD': 0.15
        }
    else:
        volatilities_usd = {
            'EUR': 0.12,
            'JPY': 0.15,
            'GBP': 0.12,
            'CHF': 0.2,
            'CAD': 0.16,
            'AUD': 0.14,
            'NZD': 0.2
        }
    
    correlation_matrix_usd = np.array([
        [1.0, 0.3, 0.2, 0.1, 0.1, 0.2, 0.1],
        [0.3, 1.0, 0.4, 0.2, 0.2, 0.1, 0.2],
        [0.2, 0.4, 1.0, 0.3, 0.3, 0.2, 0.3],
        [0.1, 0.2, 0.3, 1.0, 0.4, 0.3, 0.2],
        [0.1, 0.2, 0.3, 0.4, 1.0, 0.2, 0.1],
        [0.2, 0.1, 0.2, 0.3, 0.2, 1.0, 0.4],
        [0.1, 0.2, 0.3, 0.2, 0.1, 0.4, 1.0]
    ])
    return volatilities_usd, correlation_matrix_usd

# Functions to calculate PFE
def convert_to_usd(notional, currency, spot_rate):
    if currency == 'USD':
        return notional
    return notional / spot_rate

# Create an exposure vector
def create_exposure_vector(buy_currency, sell_currency, buy_notional_usd, sell_notional_usd, volatilities_usd):
    currencies = list(volatilities_usd.keys())
    exposures = np.zeros(len(currencies))

    for i, currency in enumerate(currencies):
        if currency == buy_currency:
            exposures[i] += buy_notional_usd
        if currency == sell_currency:
            exposures[i] -= sell_notional_usd
    return exposures

# Create an exposure vector
def create_aggregated_exposure_vector(aggregated_exposures, volatilities_usd):
    currencies = list(volatilities_usd.keys())
    exposures = np.zeros(len(currencies))

    for currency, notional in aggregated_exposures.items():
        if currency in currencies:
            index = currencies.index(currency)
            exposures[index] = notional
    return exposures


def calculate_pfe_with_cov_matrix(exposures, days, cov_matrix, confidence_level=0.99):
    # ensure that the length of exposure is equal to a side of the covariance matrix
    if len(exposures) != cov_matrix.shape[0]:
        raise ValueError("Length of exposures should match the covariance matrix")
    
    exposures_vector = np.array(exposures)
    # adjust the covariance matrix for the number of days, the covariance matrix is assumed to be annualized
    cov_matrix = cov_matrix * days / 365
    
    variance = exposures_vector.T @ cov_matrix @ exposures_vector
    stddev = np.sqrt(variance)
    z_score = norm.ppf(confidence_level)
    
    pfe = stddev * z_score
    return pfe


def calculate_pfe_for_trade(row, volatilities, cov_matrix, time_grid, mpor):
    pfe_results = {}
    buy_currency = row['BuyCurrency']
    sell_currency = row['SellCurrency']
    spot_rate = row['SpotRate']
    buy_notional_usd = convert_to_usd(row['BuyNotional'], buy_currency, spot_rate)
    sell_notional_usd = convert_to_usd(row['SellNotional'], sell_currency, spot_rate)
    
    for days in time_grid:
        if row['MaturityDate'] < row['TradingDate'] + timedelta(days=days):
            continue  # Exclude matured trades

        exposures = create_exposure_vector(buy_currency, sell_currency, buy_notional_usd, sell_notional_usd, volatilities)
        
        standalone_pfe = calculate_pfe_with_cov_matrix(exposures, days, cov_matrix, confidence_level=0.99)
        collateralized_pfe = calculate_pfe_with_cov_matrix(exposures, mpor, cov_matrix, confidence_level=0.99)
        
        pfe_results[days] = {
            'Standalone': standalone_pfe,
            'Collateralized': collateralized_pfe
        }
    
    return pfe_results

def aggregate_exposures_by_counterparty(transactions_df):
    counterparty_exposures = {}
    
    for _, row in transactions_df.iterrows():
        buy_currency = row['BuyCurrency']
        sell_currency = row['SellCurrency']
        spot_rate = row['SpotRate']
        buy_notional_usd = convert_to_usd(row['BuyNotional'], buy_currency, spot_rate)
        sell_notional_usd = convert_to_usd(row['SellNotional'], sell_currency, spot_rate)
        counterparty = row['Counterparty']
        
        if counterparty not in counterparty_exposures:
            counterparty_exposures[counterparty] = {}
        
        if buy_currency not in counterparty_exposures[counterparty]:
            counterparty_exposures[counterparty][buy_currency] = 0
        if sell_currency not in counterparty_exposures[counterparty]:
            counterparty_exposures[counterparty][sell_currency] = 0
            
        counterparty_exposures[counterparty][buy_currency] += buy_notional_usd
        counterparty_exposures[counterparty][sell_currency] -= sell_notional_usd
    
    return counterparty_exposures

def calculate_pfe_for_counterparties(counterparty_exposures, volatilities, cov_matrix, time_grid, mpor):
    counterparty_pfe = {}
    
    for counterparty, exposures in counterparty_exposures.items():
        exposure_vector = create_aggregated_exposure_vector(exposures, volatilities)
        pfe_results = {}
        
        for days in time_grid:
            standalone_pfe = calculate_pfe_with_cov_matrix(exposure_vector, days, cov_matrix, confidence_level=0.99)
            collateralized_pfe = calculate_pfe_with_cov_matrix(exposure_vector, mpor, cov_matrix, confidence_level=0.99)
            
            pfe_results[days] = {
                'Standalone': standalone_pfe,
                'Collateralized': collateralized_pfe
            }
        
        counterparty_pfe[counterparty] = pfe_results
    
    return counterparty_pfe



In [351]:
# Margin period of risk (MPOR) in days
mpor = 10

# Fetch volatilities and correlation matrix
volatilities_usd, correlation_matrix_usd = fetch_vol_and_correlation()

# Calculate variance-covariance matrix for USD pairs
volatility_vector = np.array(list(volatilities_usd.values()))
cov_matrix_usd = np.dot(volatility_vector[:, None], volatility_vector[None, :]) * correlation_matrix_usd

# Aggregate exposures by counterparty
counterparty_exposures = aggregate_exposures_by_counterparty(transactions_df)

# Calculate PFE for each counterparty
counterparty_pfe = calculate_pfe_for_counterparties(counterparty_exposures, volatilities_usd, cov_matrix_usd, time_grid, mpor)

# Convert counterparty PFE to DataFrame for display
counterparty_pfe_df = []
for counterparty, pfe_data in counterparty_pfe.items():
    for days, pfes in pfe_data.items():
        row = {'Counterparty': counterparty, 'Days': days, 'Standalone_PFE': pfes['Standalone'], 'Collateralized_PFE': pfes['Collateralized']}
        counterparty_pfe_df.append(row)

counterparty_pfe_df = pd.DataFrame(counterparty_pfe_df)

# Go thru each transaction_df and calculate PFE as a full term structure in a new dataframe, with the trade id as the key
pfe_results = []
for idx, row in transactions_df.iterrows():
    trade_pfe = calculate_pfe_for_trade(row, volatilities_usd, cov_matrix_usd, time_grid, mpor)
    # Append the trade ID and cpty to pfe_results to the trade_pfe, no need to flatten the dictionary
    pfe_results.append(
        {
            'TransactionID': row['TransactionID'], 
            'Counterparty': row['Counterparty'], 
            'BuyCurrency': row['BuyCurrency'],
            'SellCurrency': row['SellCurrency'],
            'MaturityDate': row['MaturityDate'],
            'PFE': trade_pfe})
    
# flatten pfe_results and PFE with the days as a new column to a dataframe
pfe_results_df = []
for row in pfe_results:
    for days, pfe in row['PFE'].items():
        pfe_results_df.append(
            {'TransactionID': row['TransactionID'], 
             'Counterparty': row['Counterparty'], 
             'BuyCurrency': row['BuyCurrency'],
             'SellCurrency': row['SellCurrency'],
             'MaturityDate': row['MaturityDate'],
             'Days': days, 
             'Standalone_PFE': pfe['Standalone'], 
             'Collateralized_PFE': pfe['Collateralized']})

pfe_results_df = pd.DataFrame(pfe_results_df)

In [352]:
# Next day

# Fetch volatilities and correlation matrix
volatilities_usd_d1, correlation_matrix_usd_d1 = fetch_vol_and_correlation(alternative=True)

# Calculate variance-covariance matrix for USD pairs
volatility_vector_d1 = np.array(list(volatilities_usd_d1.values()))
cov_matrix_usd_d1 = np.dot(volatility_vector_d1[:, None], volatility_vector_d1[None, :]) * correlation_matrix_usd_d1

# Aggregate exposures by counterparty
counterparty_exposures_d1 = aggregate_exposures_by_counterparty(transactions_df)

# Calculate PFE for each counterparty
counterparty_pfe_d1 = calculate_pfe_for_counterparties(counterparty_exposures_d1, volatilities_usd_d1, cov_matrix_usd_d1, time_grid, mpor)

# Convert counterparty PFE to DataFrame for display
counterparty_pfe_d1_df = []
for counterparty, pfe_data in counterparty_pfe_d1.items():
    for days, pfes in pfe_data.items():
        row = {'Counterparty': counterparty, 'Days': days, 'Standalone_PFE': pfes['Standalone'], 'Collateralized_PFE': pfes['Collateralized']}
        counterparty_pfe_d1_df.append(row)

counterparty_pfe_d1_df = pd.DataFrame(counterparty_pfe_d1_df)

# Go thru each transaction_df and calculate PFE as a full term structure in a new dataframe, with the trade id as the key
pfe_results_d1 = []
for idx, row in transactions_df.iterrows():
    trade_pfe = calculate_pfe_for_trade(row, volatilities_usd_d1, cov_matrix_usd_d1, time_grid, mpor)
    # Append the trade ID and cpty to pfe_results to the trade_pfe, no need to flatten the dictionary
    pfe_results_d1.append(
        {
            'TransactionID': row['TransactionID'], 
            'Counterparty': row['Counterparty'], 
            'BuyCurrency': row['BuyCurrency'],
            'SellCurrency': row['SellCurrency'],
            'MaturityDate': row['MaturityDate'],
            'PFE': trade_pfe})
    
# flatten pfe_results and PFE with the days as a new column to a dataframe
pfe_results_d1_df = []
for row in pfe_results_d1:
    for days, pfe in row['PFE'].items():
        pfe_results_d1_df.append(
            {'TransactionID': row['TransactionID'], 
             'Counterparty': row['Counterparty'], 
             'BuyCurrency': row['BuyCurrency'],
             'SellCurrency': row['SellCurrency'],
             'MaturityDate': row['MaturityDate'],
             'Days': days, 
             'Standalone_PFE': pfe['Standalone'], 
             'Collateralized_PFE': pfe['Collateralized']})

pfe_results_d1_df = pd.DataFrame(pfe_results_d1_df)

In [353]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.preprocessing import OneHotEncoder

# Assume pfe_results_df and pfe_results_d1_df are already provided

# Perform an inner join on TransactionID and Days
merged_df = pd.merge(pfe_results_df, pfe_results_d1_df, on=['TransactionID', 'Days'], suffixes=('_d0', '_d1'))

# Calculate the differences
merged_df['Standalone_PFE_Diff'] = merged_df['Standalone_PFE_d1'] - merged_df['Standalone_PFE_d0']
merged_df['Collateralized_PFE_Diff'] = merged_df['Collateralized_PFE_d1'] - merged_df['Collateralized_PFE_d0']

# Set a significant threshold based on 95th percentile of the differences

# Standalone PFE Difference
standalone_threshold = np.percentile(abs(merged_df['Standalone_PFE_Diff']), 95)

# Collateralized PFE Difference
collateralized_threshold = np.percentile(abs(merged_df['Collateralized_PFE_Diff']), 95)

# Create binary labels
merged_df['Standalone_Significant'] = (abs(merged_df['Standalone_PFE_Diff']) > standalone_threshold).astype(int)
merged_df['Collateralized_Significant'] = (abs(merged_df['Collateralized_PFE_Diff']) > collateralized_threshold).astype(int)

# One-hot encode categorical features
categorical_features = ['BuyCurrency_d0', 'SellCurrency_d0']
onehot_encoder = OneHotEncoder(sparse=False, drop='first')
encoded_features = onehot_encoder.fit_transform(merged_df[categorical_features])
encoded_feature_names = onehot_encoder.get_feature_names_out(categorical_features)

encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)

# Combine all features
final_features_df = pd.concat([merged_df[['Days']], encoded_df], axis=1)

# Define features and target for standalone PFE difference
X = final_features_df
y_standalone = merged_df['Standalone_Significant']
y_collateralized = merged_df['Collateralized_Significant']

# Train a Decision Tree for Standalone PFE Difference
tree_model_standalone = DecisionTreeRegressor(random_state=0, max_depth=3)
tree_model_standalone.fit(X, y_standalone)

# Export the tree as text for Standalone PFE Difference
tree_rules_standalone = export_text(tree_model_standalone, feature_names=final_features_df.columns.to_list())

text = ''
text += 'Decision Tree Rules for Standalone PFE Difference:\n'
text += tree_rules_standalone

# Train a Decision Tree for Collateralized PFE Difference
tree_model_collateralized = DecisionTreeRegressor(random_state=0, max_depth=3)
tree_model_collateralized.fit(X, y_collateralized)

# Export the tree as text for Collateralized PFE Difference
tree_rules_collateralized = export_text(tree_model_collateralized, feature_names=final_features_df.columns.to_list())

text += '\nDecision Tree Rules for Collateralized PFE Difference:\n'
text += tree_rules_collateralized

# print volatility and correlation matrix to string, concatenate and with headlines explaining them
volatilities_usd_text = 'Volatilities for USD pairs:\n'
volatilities_usd_text += str(volatilities_usd)
volatilities_usd_text += '\n\n'

correlation_matrix_usd_text = 'Correlation Matrix for USD pairs:\n'
correlation_matrix_usd_text += str(correlation_matrix_usd)
correlation_matrix_usd_text += '\n\n'

volatilities_usd_d1_text = 'Volatilities for USD pairs in d1:\n'
volatilities_usd_d1_text += str(volatilities_usd_d1)
volatilities_usd_d1_text += '\n\n'

correlation_matrix_usd_d1_text = 'Correlation Matrix for USD pairs in d1:\n'
correlation_matrix_usd_d1_text += str(correlation_matrix_usd_d1)
correlation_matrix_usd_d1_text += '\n\n'

# Combine all text
market_data_text = volatilities_usd_text + correlation_matrix_usd_text + volatilities_usd_d1_text + correlation_matrix_usd_d1_text


/Applications/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [360]:
merged_df

,TransactionID,Counterparty_d0,BuyCurrency_d0,SellCurrency_d0,MaturityDate_d0,Days,Standalone_PFE_d0,Collateralized_PFE_d0,Counterparty_d1,BuyCurrency_d1,SellCurrency_d1,MaturityDate_d1,Standalone_PFE_d1,Collateralized_PFE_d1,Standalone_PFE_Diff,Collateralized_PFE_Diff,Standalone_Significant,Collateralized_Significant
0,44062866,Maldonado-Roberts,EUR,USD,2025-06-13,1,3.399570e+05,1.075038e+06,Maldonado-Roberts,EUR,USD,2025-06-13,4.079484e+05,1.290046e+06,67991.403804,215007.697334,0,0
1,44062866,Maldonado-Roberts,EUR,USD,2025-06-13,2,4.807718e+05,1.075038e+06,Maldonado-Roberts,EUR,USD,2025-06-13,5.769262e+05,1.290046e+06,96154.365385,215007.697334,0,0
2,44062866,Maldonado-Roberts,EUR,USD,2025-06-13,3,5.888228e+05,1.075038e+06,Maldonado-Roberts,EUR,USD,2025-06-13,7.065874e+05,1.290046e+06,117764.565867,215007.697334,0,0
3,44062866,Maldonado-Roberts,EUR,USD,2025-06-13,4,6.799140e+05,1.075038e+06,Maldonado-Roberts,EUR,USD,2025-06-13,8.158968e+05,1.290046e+06,135982.807609,215007.697334,0,0
4,44062866,Maldonado-Roberts,EUR,USD,2025-06-13,5,7.601670e+05,1.075038e+06,Maldonado-Roberts,EUR,USD,2025-06-13,9.122004e+05,1.290046e+06,152033.400792,215007.697334,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31722,34457847,"Tran, Velazquez and Phillips",USD,CAD,2025-04-28,240,3.011449e+06,6.147095e+05,"Tran, Velazquez and Phillips",USD,CAD,2025-04-28,3.706399e+06,7.565656e+05,694949.854450,141856.045019,0,0
31723,34457847,"Tran, Velazquez and Phillips",USD,CAD,2025-04-28,270,3.194124e+06,6.147095e+05,"Tran, Velazquez and Phillips",USD,CAD,2025-04-28,3.931230e+06,7.565656e+05,737105.632000,141856.045019,0,0
31724,34457847,"Tran, Velazquez and Phillips",USD,CAD,2025-04-28,300,3.366903e+06,6.147095e+05,"Tran, Velazquez and Phillips",USD,CAD,2025-04-28,4.143880e+06,7.565656e+05,776977.557752,141856.045019,0,0
31725,34457847,"Tran, Velazquez and Phillips",USD,CAD,2025-04-28,330,3.531237e+06,6.147095e+05,"Tran, Velazquez and Phillips",USD,CAD,2025-04-28,4.346138e+06,7.565656e+05,814900.937400,141856.045019,0,0


In [361]:
final_features_df

,Days,BuyCurrency_d0_EUR,BuyCurrency_d0_GBP,BuyCurrency_d0_NZD,BuyCurrency_d0_USD,SellCurrency_d0_CHF,SellCurrency_d0_JPY,SellCurrency_d0_USD
0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
31722,240,0.0,0.0,0.0,1.0,0.0,0.0,0.0
31723,270,0.0,0.0,0.0,1.0,0.0,0.0,0.0
31724,300,0.0,0.0,0.0,1.0,0.0,0.0,0.0
31725,330,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [357]:
system_prompt = """You will be given a PFE difference tree for both collateralized and uncollateralized standalone trades between two days. 
Additionally, market data inputs such as volatility and spot rates for both days will be provided.
Note that the columns with categorical features are one-hot encoded.
Also the differences in PFE are calculated for each trade and the significant differences are identified based on a threshold, which is what the decision tree is trained on.
When you see that the number of days are affecting the PFE, it is likely to be due to volatility and correlation changes.
For FX trades that are uncollateralized, it is expected that the PFE will increase with number of days in the future, but that's not as expected for collateralized trades.

Your task is to compare the differences between the two days, analyze the key factors driving the changes, and summarize your findings. Specifically, please:

1. Identify the primary drivers for the differences in PFE. Note that many features are one-hot encoded, so you may need to interpret the rules accordingly.
2. Explain how changes in market data (e.g., volatility, spot rates) are impacting the PFE calculations.
3. Highlight any significant patterns or anomalies in the decision tree.
4. Provide recommendations on what we should examine further based on your analysis. Note that we are only interested in the computational aspects of the analysis, not the business implications.

Finally please provide an executive summary on the next steps and investigations that should be conducted based on your analysis.
"""

In [358]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "text": system_prompt,
          "type": "text"
        }
      ]
    },  
    {
        "role": "user",
        "content": [
            {
            "text": text,
            "type": "text"
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
            "text": market_data_text,
            "type": "text"
            }
        ]

    }
  ],
  temperature=1,
  max_tokens=1000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [359]:
#Parse out the response
print(response.choices[0].message.content)

#### 1. Identifying the Primary Drivers for PFE Differences

From the decision tree rules for standalone (uncollateralized) and collateralized PFE differences, we can identify key factors impacting the PFE changes:

**Standalone PFE Differences:**
- **Days to Maturity:** A significant split in the tree is based on "Days." Days <= 105 and Days > 105 are prominent nodes, with further splits occurring around Days <= 52.50, Days <= 80.50, etc.
- **Sell Currency (CHF):** Trades involving CHF as the sell currency (SellCurrency_d0_CHF) are crucial. The splits around the SellCurrency_d0_CHF feature indicate higher PFE differences when this currency is involved, especially for trades involving more than 52.50 and 105 days to maturity.
- **Buy Currency (NZD):** Further splits based on BuyCurrency_d0_NZD show differences in PFE when NZD is the buy currency. This factor differentiates trades under the condition of Days > 105.

**Collateralized PFE Differences:**
- **Sell Currency (CHF):** Similar 